# Classifier for Predicting Smoker Status 

Can we predict whether someone has been a smoker (Has Quit or Currently is) with biometric data?
I'm not sure, but let's find out.

We'll start with the smoking-drinking-dataset available here (automatically loaded to the workspace if you are viewing this on Kaggle):
https://www.kaggle.com/datasets/sooyoungher/smoking-drinking-dataset

This dataset, contributed by Kaggler Soo.Y, was collected from National Health Insurance Service in Korea. The dataset has 991,000 labeled observations and seems very clean. 

Next, we'll need some libraries so we can load the data and run the analysis.


In [ ]:
 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

 
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import precision_score
from sklearn.metrics import PrecisionRecallDisplay
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
#read in the data to a dataframe
df = pd.read_csv('/kaggle/input/smoking-drinking-dataset/smoking_driking_dataset_Ver01.csv')
df.head()

In [ ]:
print("Shape",df.shape)
print(df.describe())

## EDA - Exploratory Data Analysis
This data is pretty clean (No minimums less than 1), and sizeable (991346 rows of data).

## Preprocessing and Data Cleansing
Are there any categorical variables? Let's look at a row to see.


In [ ]:
df.iloc[0,:]

Seems we have two binary categorical variables. 
- Sex (F/M) 
- Drinker (yes/no)

Both are binary so let's encode those variables with True/False for 1s and 0s later.

At the same time, let's address our target variables of Smoking.
- SMK_stat_type_cd = Smoking state, 1(never), 2(used to smoke but quit), 3(still smoke)
- We'll encode this as 0 never smoked, 1 has or is a smoker

In [ ]:
df["female"] = pd.get_dummies(df['sex'],1).iloc[:,0]

for i in range(df.shape[0]):
    if df['female'][i] == False:
        df['female'][i] = 0
    else:
        df['female'][i] = 1
        
df.iloc[:,[0,-1]]

In [ ]:
df["drinker"] = pd.get_dummies(df['DRK_YN'],0).iloc[:,1]
for i in range(df.shape[0]):
    if df['drinker'][i] == False:
        df['drinker'][i] = 0
    else:
        df['drinker'][i] = 1
        

df.iloc[:,[-3,-1]]


In [ ]:
df['smokers'] = df['SMK_stat_type_cd'] - 1 #sets ones to zero, twos to ones, and threes to twos
df['smokers'].loc[df['smokers'] > 1  ] = 1 #changes twos to ones
df['smokers'].describe() #mean shows us how many smokers in dataset

In [ ]:
# Check for NaNs
for i in df.columns:
    print(i,":",df[i].isna().sum())

In [ ]:
df.drop('SMK_stat_type_cd', axis=1, inplace=True)
df.drop("DRK_YN", axis=1, inplace=True)
df.drop("sex", axis=1, inplace =True)
df.columns

Now we have all numerical data, a coded target variable, and cleaned up dataframe without NaNs.

Our dataset isn't totally balanced, we see that 39.2% of respondents are or were smokers, and the remainder never started.

## Visualize Correlations
Let's move onto visualize and model the data. First we'll run a correlation plot to see how bad the multicollinearity is on this data.

In [ ]:
plt.figure(figsize=(16, 6))
heatmap = sns.heatmap(df.corr(method='pearson'), annot=True)
heatmap.set_title('Pairwise Correlation Heatmap', fontdict={'fontsize':6}, pad=12);

Well, 
- there are some highly correlated columns depending up a person's sex.
- height is highly correlated to weight.
- waistline is highly correlated to weight.
- Systolic Blood Pressures is highly correlated to Diastolic Blood Pressure.
- LDL Cholestoral is highly correlated to total cholestoral.
- SGOT_AST(Glutamate-oxaloacetate transaminase)(Aspartate transaminase) correlates highly with (Alanine transaminase).

This is expected in all but the sex category. 
- The sex of a person is descriptive. 
- The rest of these are examples of measuring similar attributes differently.

This presents options: 
1. We'll have to progress through forward and backward selection to see which ones present collinearity issues in a linear regression
2. Or we can see if decision trees do a good job with the data first 
- A: Base trees will be prone to overfitting
- B: ADA Boost will be better than Base Trees
- C: Random Forest will be able to handle the collinearity and pull patterns across collinearity.

Let's try # 2C first.

## Build & Train a Random Forest Classifier

In [ ]:
Xtr,Xte, ytr, yte = train_test_split(df.iloc[:,0:-2],df.iloc[:,-1],test_size=0.2,shuffle=True,random_state =12)
print(Xtr.head())

print("Shapes (train):",Xtr.shape,ytr.shape)
print("Shapes (test):", Xte.shape,yte.shape)

For documentation sake, what are the Random Forest Model parameters?

In [ ]:
rf = RandomForestClassifier(random_state = 12)

from pprint import pprint

# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(rf.get_params())

Okay, a few things stay a few things to change. Gini Coefficient is good and will stay (detecting purity of the splits) and a square root of the number of features will help the classifier mitigate the collinearity. 

Moving on to things we'll change, we want to use stumps (max depth 2), expect a few samples per leaf (we'll say 12 to start), want to run the job in parallel (n_jobs = -1), want to bootstrap the data for randomization (so central limit theorem helps us avoid overfitting), and a whole bunch of trees (say 1200). 

We set those parameters, train the model with the training data, and then predict both the test and train outputs with our new model. Verbose is set to 1 so we will get to see some of the progress.

In [ ]:
class1rf = RandomForestClassifier(n_estimators=212,  criterion='gini',max_features='sqrt',
                                  bootstrap=True, max_depth=2, min_samples_leaf = 12, n_jobs=-1,
                                 verbose=1, ccp_alpha = .00) 

#fit the classifier to training data
class1rf.fit(Xtr,ytr)                                  
print("Random Forest fit. \n Predicting Train Values..." )

#predict the training dataset
yarf = class1rf.predict(Xtr)

#predict the test dataset
yarf2 = class1rf.predict(Xte)                              

## Score the Initial Random Forest Classifier
With our fit training data, and predicted train and test set data, let's score the model.

### Numerical Scores

In [ ]:
C1Trroc = roc_auc_score(ytr,yarf)
C1Teroc = roc_auc_score(yte,yarf2)

C1Trps = precision_score(ytr,yarf)
C1Teps = precision_score(yte,yarf2)

C1Trrs = recall_score(ytr,yarf)
C1Ters = recall_score(yte,yarf2)

C1Trf1 = 2*(C1Trrs * C1Trps)/(C1Trrs + C1Trps)
C1Tef1 =  2*(C1Ters * C1Teps)/(C1Ters + C1Teps)

scoring = pd.DataFrame({
             "model" : ["Train", "Test"],
             "Precision" : [C1Trps,C1Teps],
             "Recall" : [C1Trrs,C1Ters],
             "F1 Score" : [C1Trf1,C1Tef1],
             "ROC-AUC": [C1Trroc,C1Teroc]
             
             })
scoring 

### Visual Score Graphics

In [ ]:
C1Trfp,C1Trtp, _ = roc_curve(ytr,yarf)
C1Tefp,C1Tetp, _ = roc_curve(yte,yarf2)
#Training ROC
plt.figure(figsize=[8,5])
plt.subplot(121)
plt.fill_between(C1Trfp,C1Trtp)
plt.ylabel("True Positive")
plt.xlabel("False Positive")
plt.title("Train ROC Curve");

plt.subplot(122)
#Testing ROC 
plt.fill_between(C1Tefp,C1Tetp)

plt.xlabel("False Positive")
plt.title("Test ROC Curve");
plt.show()
print("Train AUC:", C1Trroc ,"\t \t Test AUC:" , C1Teroc )

In [ ]:
C1Trprec,C1Trrecall, _ = precision_recall_curve(ytr,yarf)
C1Teprec,C1Terecall, _ = precision_recall_curve(yte,yarf2)
#Training Precision Recall Curve
plt.figure(figsize=[8,5])
plt.subplot(121)
plt.fill_between(C1Trrecall,C1Trprec)
plt.ylabel("Precision")
plt.xlabel("Recall")
plt.title("Train Precision-Recall curve");

plt.subplot(122)
#Training Precision Recall Curve
plt.fill_between(C1Trrecall,C1Trprec)
#plt.ylabel("Precision")
plt.xlabel("Recall")
plt.title("Test Precision-Recall curve");
plt.show()

### Initial Classifier Analysis
These results aren't bad, we are over 81% on recall, 70% on precision, and have F1 scores in the .75 area for both train and test sets. This tells us we have probably not overfit our model because the metrics are similar whether the model has seen the data (training set) or is new data (test set). Moreover, area under the ROC-AUC and Precision Recall Curves appears to illustrate identical shapes and more area identified than blank. We have succussfully fit a model in under 10 minutes, tested, scored, and completed our cursory analysis. 

However, we haven't tuned this model at all. The hyperparameters we chose were just guesses. The cursory analysis does show that this method is promising, but to deploy a model like this we'd like better scores. Therefore, the value of the cursory model justifies investing time to tune the model. Let's see if we can increase those scores by tuning the hyperparameters.

## Tune Random Forest Classifier Hyperparameters
Here, we'll use the gridsearchCV method to tune our random forest.
- Start by choosing the parameters for the grid.
- Train the data on each of the sets of parameters.
- Select the best set of parameters.
- Predict train outcomes and then test outcomes.
- Run metrics on those outcomes to gauge efficacy, overfitting, and final recommendation.


In [ ]:
# Build the grid of options 
grids = {
        'n_estimators':[int(N) for N in np.linspace(100,1000,3)],
        'max_depth': [2,3,4,5]
        #'min_samples_split': [.1,.01,.05],
        #'min_samples_leaf': [2,27,52,104],   
        #'max_samples' : [ int(N) for N in np.linspace(1000,12000,6)] 
        }
# Let people see the output of this step...
grids

In [ ]:
# run grid search on random forest classifier, with grid options, 
# on a single proc (the other 3 are for RandomForest to run full blast),
# with 3 fold validation, verbose output (3), scoring based on f1 score,
# and raise an error if score is nan.
hypertesting = GridSearchCV(class1rf, 
                            param_grid=grids, n_jobs=1, 
                            cv= 3, verbose = 3, scoring='f1',
                            error_score ='raise'  ).fit(Xtr,ytr)

### Table of F1 Scores and Parameters

In [ ]:
seee = pd.DataFrame(hypertesting.cv_results_)
seee

### Combine Scores With Initial Classifer Results

In [ ]:
#predict the test set with optimized model
preds = hypertesting.predict(Xte)
 
#score the test set (ROC-AUC, Precision, Recall, & F1)
C2Teroc = roc_auc_score(yte,preds)
 
C2Teps = precision_score(yte,preds)
 
C2Ters = recall_score(yte,preds)
 
C2Tef1 =  2*(C2Ters * C2Teps)/(C2Ters + C2Teps)

#add scores to dataframe of metrics above for analysis
scoring.loc[len(scoring.index)] = ['GridSearch + Test Set', C2Teps, C2Ters, C2Tef1,C2Teroc] 
scoring

### Analysis

The GridSearch returned a best F1 score of 78.4% on the training data. The dataframe above shows that the F1 Score on the unseen test set is even higher at 78.53%. This leads us to believe that the model is not overfit. 

Additionally, the optimized model results in a 2.5% increase in F1 scores versus the untuned model (76% on test set vs 78.5% on test set). This is a substantial increase, and warrants a look under the hood. Further analysis shows that the Recall of the tuned model increased 7 points, and the Precision stayed roughly the same (a loss of .5% in precision). The extra hour of processing time to run the grid search did increase the value of the model.

### Visual Score Graphics
Before we go, we'll generate the ROC and Precision-Recall curves of the intial testset results and the final test set results.

In [ ]:
C2Tefp,C2Tetp, _ = roc_curve(yte,preds) 

plt.figure(figsize=[8,5])
plt.subplot(121)
#Initial Test ROC 
plt.fill_between(C1Tefp,C1Tetp)

plt.xlabel("False Positive")
plt.title("Test ROC Curve");

#Grid Test ROC
plt.subplot(122)
plt.fill_between(C2Tefp,C2Tetp)
plt.ylabel("True Positive")
plt.xlabel("False Positive")
plt.title("Grid Test ROC Curve");


plt.show()
print("Test AUC:", C1Teroc ,"\t \t GridSearch Test AUC:" , C2Teroc )

C2Teprec,C2Terecall, _ = precision_recall_curve(yte, preds) 

 
plt.figure(figsize=[8,5])
plt.subplot(121)
#Initial Testing Precision Recall Curve
plt.fill_between(C1Trrecall,C1Trprec)
#plt.ylabel("Precision")
plt.xlabel("Recall")
plt.title("Test Precision-Recall curve")


plt.subplot(122)
# Final Testing Precision Recall Curve
plt.fill_between(C2Terecall,C2Teprec)
plt.ylabel("Precision")
plt.xlabel("Recall")
plt.title("Grid Search Test Precision-Recall curve");
plt.show()

### Final Analysis of Classifier
Yes, there is appreciable improvment in the output graphics for ROC and Precision-Recall. But they are certainly not earth shattering. I would agree that there is more tuning to be done. We could certainly continue to tune this model. Viable reasons include a competition or because a client was willing to pay for it. However, the initial grid search tells us that we have reached diminishing returns on this particular model. Without feature engineering, different data, or other manipulation of the setup, this model is probably within 10% of the final results. 

Here, we revert to the initial research question, and deliver our results.

# Answer Initial Research Inquiry
Can we determine if a person was a smoker based on this dataset?
- The answer is yes, with a 90% recall rate and 69% precision on those predictions. 
- In other words our model can find 9/10 of the smokers, and is accurate in 7/10 predictions.

The only thing left to determine are the indicators the model is using for these predictions. 

How is the model making these decisions?

# Explain the Model
We'll use the shapely values to illustrate how the model thinks on three random cases. 
- First we'll build the explainer on the gridsearch best model.
- Then we'll output a graphic representation (forceplot) of why the model returned the probability for each case.
- And finally check whether the model categorized each case correctly.

## Granular Model Decision Analysis 
### Shap Force Plot1 - Observation 312

In [ ]:
import shap  
# Create Tree Explainer object that can calculate shap values
explainer = shap.TreeExplainer(hypertesting.best_estimator_)

In [ ]:
# Calculate Shap values on a random instance
chosen_instance = Xte.iloc[312,:]
shap_values = explainer.shap_values(chosen_instance)
shap.initjs()
shap.force_plot(explainer.expected_value[1], shap_values[1], chosen_instance)

Here we see that the most significant factor in the non-smoking classification is that the subject is female. Hemoglobin, height, weight, serum creatine score, and waistline size also contribute to the 0 class score. However, triglyceride levels and gamma_GTP scores keep this classification from being a perfect zero log-odds. The model log odds for a smoker are .12, which is less than the .5 decision boundary between the classes. The model classifies this as a non-smoker.


In [ ]:
print(f"The actual value from the test set for record 312 is {yte.iloc[312]}, which corroborates the model.")

### Shap Force Plot2 - Observation 712

In [ ]:
# Calculate Shap values on a random instance
chosen_instance = Xte.iloc[712,:]
shap_values = explainer.shap_values(chosen_instance)
shap.initjs()
shap.force_plot(explainer.expected_value[1], shap_values[1], chosen_instance)

Here we see that the most significant factor in the smoking classification is that the subject is male. Height, gamma_GTP, and triglyceride levels contribute to that score in descending importance. The age of 40 is significant, but not as much so. Hemoglobin and weight characteristics of the subject reduce the log-odds score, but not below the threshold for a decision. The model log odds for a smoker are .67, which is more than the .5 decision boundary for the model. So the model classifies this as a smoker.

In [ ]:
print(f"The actual value from the test set is {yte.iloc[712]}, which corroborates the model.")

### Shap Force Plot3 - Observation 1212

In [ ]:
# Calculate Shap values on a random instance
chosen_instance = Xte.iloc[1212,:]
shap_values = explainer.shap_values(chosen_instance)
shap.initjs()
shap.force_plot(explainer.expected_value[1], shap_values[1], chosen_instance)

Here, again, we see that the most significant factor in the non-smoking classification is that the subject is female. Hemoglobin, Gamma_GTP, height, serum creatine levels and weight contribute to the 0 class score in descending importance. However, hemoglobin levels keep this classification from being a perfect zero log-odds. The model log odds for a smoker are .16, which is less than the .5 decision boundary between the classes. The model classifies this as a non-smoker.


In [ ]:
print(f"The actual value from the test set is {yte.iloc[1212]}, which corroborates the model.")

## Model Feature Importance - Summary Plot

In [ ]:
#run shap_values on whole test set for summary plot
vals = explainer.shap_values(Xte)
shap.summary_plot(vals,  max_display = 22, feature_names=Xte.columns)

The summary plot above tells us which features have the most importance in the model (gender is greatest). Since the Random Forest of decision trees has many contradictory decision points (based on the values in adjacent features for any given observation) this type of plot is one of the most powerful ways of delineating what features contribute heavily to a classification of 1 or 0.

# Conclusion 
We can see who is or was a smoker based on their vital signs with well better than 50% guessing accuracy, (69.4%) and we can find 90% of those smokers with the model. This is good. 

However, this model doesn't have a high enough F1 score (over 80% precision and recall) to prescribe outcomes in an automated fashion.

Thanks for taking a moment to look at this notebook.
